## Problem Set 2: Merging and regular expressions

**Total points (without extra credit)**: 30 

**Background on the policy context**: here, we're going to use two datasets to practice reshaping, merging, and regular expression patterns. Both datasets relate to the broader issue of which employers might be violating the rights of temporary guestworkers granted visas under the H-2A program. Here are some articles about potential exploitation of guestworkers by firms and inequality caused by minimal oversight:

- News media coverage of labor abuses of temporary guestworkers: https://www.buzzfeednews.com/article/kenbensinger/the-pushovers 
- GAO report on labor abuses of temporary guestworkers: https://www.gao.gov/products/gao-15-154

The following datasets are located in `pset2_inputdata` (need to unzip): 

- `jobs_clean`: a dataset of guestworker jobs posted by many employers, some of whom have been debarred (banned) from the program for labor abuses; others not debarred
- `debar`: a dataset of employers who committed violations of labor regulations meant to protect temporary guestworkers 


You can view a codebook here: https://docs.google.com/spreadsheets/d/1rF9GJEC8pPKxipD0TsoG9DVdqz3EJ-b-BHEtyioAX7I/edit?usp=sharing


In [1]:
## helpful packages
import pandas as pd
import numpy as np
import random
import re
import os

## repeated printouts
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"


# 1. Reshaping data (13 points total)

Load the following dataset stored in `pset2_inputdata`: `debar.csv`

This represents employers temporarily banned from hiring workers (debar.csv); call this `debar`


View the head()


In [5]:
## your code here
debar = pd.read_csv('pset2_inputdata/debar.csv')

In [7]:
debar.head()
debar.shape
debar.info()

,Name,"City, State",Violation,Duration,Start date,End date
0,J&J Harvesting,"Leads, ND",Failure to respond to audit (partial response),2 years,1/19/2014,1/18/2016
1,"Stahlman Apiaries, Inc","Selby, SD",Failure to respond to audit (partial response),1 year,2/19/2015,2/14/2016
2,Trust Nursery,"Pulaski, NY",Failure to respond to audit (partial response),1 year,3/21/2014,3/20/2015
3,Anton Fertilizer Inc.,"Dighton, KS",Failure to respond to audit (no response),2 years,3/30/2014,3/29/2016
4,"Great Plains Fluid Service, Inc.","Greensburg, KS",Failure to respond to audit (no response),2 years,3/30/2014,3/29/2016


(114, 6)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 114 entries, 0 to 113
Data columns (total 6 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   Name         114 non-null    object
 1   City, State  113 non-null    object
 2   Violation    114 non-null    object
 3   Duration     114 non-null    object
 4   Start date   114 non-null    object
 5   End date     114 non-null    object
dtypes: object(6)
memory usage: 5.5+ KB


## 1.1 (1 point)

Print the number of rows in `debar` versus the number of unique employer names (`Name`). Is there one row per employer or multiple rows for some employers?

In [10]:
## your code here
num_rows = debar.shape[0]
num_unique_employers = debar['Name'].nunique()

print("number of rows:", num_rows)
print("number of employers:", num_unique_employers)

number of rows: 114
number of employers: 98


## 1.2 Investigating duplicated rows (2 points)

A. Create a new column in `debar`--`is_repeated`-- that tells us whether an employer (`Name`) is repeated > 1 times

*Hint*: there are multiple ways to solve this but some possibilities to get the list of names that are repeated are:
- Using value_counts() on the `Name` variable and extracting the index from that value counts 
- Using groupby to count the rows attached to one name

B. Print the rows where `is_repeated == True` and interpret

C. Subset to the rows where `is_repeated == True` and save that data as `mult_debar`. Print the head() and shape

In [13]:
## your code here
repeated_names = debar['Name'].value_counts()[debar['Name'].value_counts() > 1].index
debar['is_repeated'] = debar['Name'].isin(repeated_names)
debar.head()

,Name,"City, State",Violation,Duration,Start date,End date,is_repeated
0,J&J Harvesting,"Leads, ND",Failure to respond to audit (partial response),2 years,1/19/2014,1/18/2016,False
1,"Stahlman Apiaries, Inc","Selby, SD",Failure to respond to audit (partial response),1 year,2/19/2015,2/14/2016,False
2,Trust Nursery,"Pulaski, NY",Failure to respond to audit (partial response),1 year,3/21/2014,3/20/2015,False
3,Anton Fertilizer Inc.,"Dighton, KS",Failure to respond to audit (no response),2 years,3/30/2014,3/29/2016,False
4,"Great Plains Fluid Service, Inc.","Greensburg, KS",Failure to respond to audit (no response),2 years,3/30/2014,3/29/2016,False


In [39]:
repeated_debar = debar[debar['is_repeated'] == True]
repeated_debar

,Name,"City, State",Violation,Duration,Start date,End date,is_repeated
6,Annabella Land & Cattle,"Annabella, UT",Non Payment,1 year,5/9/2014,5/9/2015,True
7,Autumn Hill Orchard,"Groton, MA",Failure to respond to audit (no response),2 years,7/6/2014,7/5/2016,True
8,"Caddo Creek Ranch, dba Paradise Ranch","Caddo, TX",Failure to respond to audit (partial response),2 years,7/20/2014,7/19/2016,True
11,Loewen Harvesting LLC,"Brownsville, TX",Failure to respond to audit (partial response),1 year,8/20/2014,8/19/2015,True
12,Rollo Farm Labor Contractor,"Miami, FL",Failure to respond to audit (no response),2 years,8/23/2014,8/22/2016,True
14,Sharon Mathis,"Tifton, GA",Failure to respond to audit (no response),2 years,11/16/2014,11/15/2016,True
15,SRT Farms,"Morton, TX",Failure to respond to audit (no response),2 years,11/16/2014,11/15/2016,True
16,Mark Duncan,"Roosevelt, UT",Failure to respond to audit (no response),2 years,11/16/2014,11/15/2016,True
17,"Maple Ridge Custom Services, LLC","Altheimer, AK",Failure to respond to audit (partial response),2 years,11/16/2014,11/15/2016,True
18,F&W Farms,"Ingalls, KS",Failure to respond to audit (partial response),2 years,12/10/2014,12/9/2016,True


In [17]:
mult_debar = debar[debar["is_repeated"] == True]
mult_debar.head()
mult_debar.shape

,Name,"City, State",Violation,Duration,Start date,End date,is_repeated
6,Annabella Land & Cattle,"Annabella, UT",Non Payment,1 year,5/9/2014,5/9/2015,True
7,Autumn Hill Orchard,"Groton, MA",Failure to respond to audit (no response),2 years,7/6/2014,7/5/2016,True
8,"Caddo Creek Ranch, dba Paradise Ranch","Caddo, TX",Failure to respond to audit (partial response),2 years,7/20/2014,7/19/2016,True
11,Loewen Harvesting LLC,"Brownsville, TX",Failure to respond to audit (partial response),1 year,8/20/2014,8/19/2015,True
12,Rollo Farm Labor Contractor,"Miami, FL",Failure to respond to audit (no response),2 years,8/23/2014,8/22/2016,True


(32, 7)

## 1.3 Reshape mult_debar to wide to begin filtering out duplicates (4 points)

You want to separate out two cases:

- Cases where the repeat rows for one employer are due to duplicated data 
- Cases where the repeat rows for one employer represent repeated violations for different issues

There are various ways to check duplicates in this data (eg converting `Violation` to lowercase; replacing spelled-out states with two-dig state codes)

We're going to use the simple rule of:

- A row is a duplicate if, within an employer (defined by Name + City, State), the Start date for each row's violation is the same 

To begin to check this, reshape `mult_debar` to a wide dataframe (`mult_debar_wide`) with the following columns, treating the `Name` and `City, State` as the index for the pivot:

- Name
- City, State
- start_date_viol1
- start_date_viol2

Print the head and shape

In [44]:
## your code here
mult_debar.loc[:, 'Start date'] = pd.to_datetime(mult_debar['Start date'], errors='coerce')

mult_debar.loc[:, 'Violation'] = mult_debar.groupby(['Name', 'City, State']).cumcount() + 1

mult_debar_wide = mult_debar.pivot_table(
    index=['Name', 'City, State'],
    columns='Violation',
    values='Start date',
    aggfunc='first').reset_index()

mult_debar_wide.columns = ['Name', 'City, State'] + [f'start_date_viol{i}' for i in mult_debar_wide.columns[2:]]
mult_debar_wide.head()
mult_debar_wide.shape

,Name,"City, State",start_date_viol1,start_date_viol2
0,Annabella Land & Cattle,"Annabella, UT",2014-05-09 00:00:00,NaN
1,Annabella Land & Cattle,"Annabella, Utah",2014-05-09 00:00:00,NaN
2,Autumn Hill Orchard,"Groton, MA",2014-07-06 00:00:00,2014-07-06 00:00:00
3,"Caddo Creek Ranch, dba Paradise Ranch","Caddo, TX",2014-07-20 00:00:00,NaN
4,"Caddo Creek Ranch, dba Paradise Ranch","Caddo, Texas",2014-07-20 00:00:00,NaN


(19, 4)

## 1.4 Filter out duplicates from original debar data (6 points)

A. Using `mult_debar_wide`, add a column `is_dup` that takes value of True for cases where start_date_viol1 == start_date_viol2 marking the row as a duplicate

B. Going back to the original long-format data you loaded at the beginning- `debar`
    - For employers where `is_dup == True` as indicated by your wide-format dataframe, only keep `violnum == viol1`
    - For all other employers (so is_dup == False and ones we didnt need to check duplicates for), keep all violnum
    - Remove the `is_repeated` column from the `debar` data

**Hint**: you can complete part B without a for loop; `pd.concat` with axis = 0 (row binding) is one way

Call the resulting dataframe `debar_clean` and print the shape and # of unique employer names

In [47]:
## your code here
mult_debar_wide['is_dup'] = mult_debar_wide['start_date_viol1'] == mult_debar_wide['start_date_viol2']

In [49]:
duplicates = mult_debar_wide[mult_debar_wide.is_dup == True]['Name'].unique()
debar_duplicates = debar[debar['Name'].isin(duplicates) & (debar['Violation'] == 1)]
debar_non_duplicates = debar[~debar['Name'].isin(duplicates)]

debar_clean = pd.concat([debar_duplicates, debar_non_duplicates], axis=0)  # Combine two dataframes

debar_clean.shape
print("Number of unique employer names:", debar_clean['Name'].nunique())

(98, 7)

Number of unique employer names: 90


# 2. Merging and regex (17 points total)



## 2.1 Load data on job postings

The previous dataset contains a small subset of employers who faced temporary bans due to violations of H-2A program regulations

Since most of the bans have expired, we're going to see which of those employers posted new H-2A jobs in the first quarter of 2021 

Loading the `jobs_clean.csv` data stored in `pset2_inputdata`

In [60]:
# your code here to load the data 

os.getcwd()

'/Users/catherineseok/Documents/GitHub/QSS20_F24/my_homework/pset2'

In [62]:
jobs = pd.read_csv("pset2_inputdata/jobs.csv")

##  2.2 Try inner join on employer name  (2 points)

- Use the `EMPLOYER_NAME` field of the `jobs` dataset
- Use the `Name` field of the `debar_clean` dataset 

A. Use pd.merge with an inner join on those fields to see whether there are any exact matches. 

B. If there are exact matches, print the row(s) with exact matches



In [65]:
## your code here

inner = pd.merge(jobs, debar_clean, left_on = "EMPLOYER_NAME", right_on = "Name", how = "inner")
inner

,CASE_NUMBER,CASE_STATUS,RECEIVED_DATE,DECISION_DATE,TYPE_OF_EMPLOYER_APPLICATION,H2A_LABOR_CONTRACTOR,NATURE_OF_TEMPORARY_NEED,EMERGENCY_FILING,EMPLOYER_NAME,TRADE_NAME_DBA,...,EMAIL_TO_APPLY,WEBSITE_TO_APPLY,TOTAL_ADDENDUM_A_RECORDS,Name,"City, State",Violation,Duration,Start date,End date,is_repeated
0,H-300-20287-876656,Determination Issued - Certification,2020-10-20 09:20:32.010,2020-11-09 00:00:00.000,Individual Employer,Y,Seasonal,Y,Rafael Barajas,NaN,...,NaN,https://seasonaljobs.dol.gov/,7,Rafael Barajas,"Sebring, Florida",Non-payment of certification fee,1 year,9/23/2016,9/22/2017,False


## 2.3 Targeted regex (10 points total)

You want to see if you can increase the exact match rate with some basic cleaning of each 
of the employer name fields in each dataset 

### 2.3.1 Converting to upper (2 points)

A. Convert the `EMPLOYER_NAME` and `Name` fields to uppercase using list comprehension rather than df.varname.str.upper() (it's fine to do a separate list comprehension line for each of the two columns)

B. Print a random sample of 15 values of each result

C. Assign the full vector of uppercase names back to the original data, writing over the original `EMPLOYER_NAME` and `Name` columns 


In [69]:
## insert your code to turn into uppercase here

jobs_upper = [name.upper() for name in jobs.EMPLOYER_NAME if isinstance(name, str)]
debar_clean_upper = [name.upper() for name in debar_clean.Name if isinstance(name, str)]

In [71]:
## insert your code for the random sample

random.sample(jobs_upper, 15)
random.sample(debar_clean_upper, 15)

['NORTH TEXAS FARMS, INC.',
 'MANZANA LLC',
 'NICK SNODGRASS FARMS',
 'WILLIS HANSEN',
 'AB&T ZAUNBRECHER FARMS',
 'MITCH BENSON FARMS',
 'WENAS HOP FARM, LLC',
 'NICHOLAS P. VONDENSTEIN',
 'BENJAMIN M RAMIREZ HARVESTING, INC.',
 'BONNIE PLANTS, INC.',
 'AGRICO LABOR LLC',
 'KEYSTONE FORESTRY SERVICES',
 "ROWLEY'S SOUTHRIDGE FARMS",
 'MCFADDIN LAND & CATTLE CO, LLC',
 'A. OSEGUERA COMPANY, INC.']

['REIMER’S INC.',
 'JAVIER XISTO ZAPATA',
 'ROBERT D. TOWLES',
 'SAXTONS RIVER ORCHARDS, INC.',
 'TODD DAULTON',
 'RB FARM PICKING, INC',
 'YOLANDA CHAVEZ FARMING',
 'CISCO PRODUCE INC.',
 'OMEGA LAMB, LLC',
 'TRAVIS AND TARA LAMBOURN',
 'JESUS LEDESMA',
 'OLD TREE FARMS/VERPAALEN CUSTOM SERVICE',
 'LOUIE M. ASUMENDI',
 'YOLANDA CHAVEZ',
 'CITY PINESTRAW AND HARVESTING']

In [73]:
## insert your code for assigning the uppercase names back to the data

jobs.EMPLOYER_NAME = jobs_upper
debar_clean.Name = debar_clean_upper

### 2.3.2 Cleaning up punctuation (4 points)

You notice that INC, CO, and LLC are sometimes followed by a period (.) but sometimes not

A. For each dataset, write a regex pattern using `re.sub` to remove the . but only if it's preceded by INC, LLC, or CO 

Make sure LLC, INC, CO remain part of the string but just without the dot

B. Test the pattern on the positive and negative example we provide below and print the result. See the Github issue for examples of what to return


**Hint**: https://stackoverflow.com/questions/7191209/python-re-sub-replace-with-matched-content



In [76]:
pos_example_1 = "CISCO PRODUCE INC."
pos_example_2 = "AVOYELLES HONEY CO., LLC"
neg_example = "E.V. RANCH LLP"

In [78]:
## insert your code here with the regex pattern for part A

remove_period_pattern = r"(INC|CO|LLC)\."

## insert your code to use re.sub to apply the pattern to the test cases for part B

example_list = [pos_example_1, pos_example_2, neg_example]
trying_no_period = [re.sub(remove_period_pattern, r"\1", company) for company in example_list]
trying_no_period

['CISCO PRODUCE INC', 'AVOYELLES HONEY CO, LLC', 'E.V. RANCH LLP']

### 2.3.3 (4 points)

Use that pattern in conjunction with `re.sub` and list comprehension to clean the employer name columns in each dataset. Save the new columns as `name_clean` in each. Then, use row subsetting to (1) subset to rows that changed names and (2) for:

- `debar_clean` print the `Name` and `name_clean` columns
- `jobs` print the `EMPLOYER_NAME` and `name_clean` columns

Make sure to use the uppercase versions of the variables


In [81]:
## your code here to clean the columns

def clean_name(company_name):
    return re.sub(remove_period_pattern, r"\1", company_name)

debar_clean["name_clean"] = debar_clean["Name"].apply(clean_name)
jobs["name_clean"] = jobs["EMPLOYER_NAME"].apply(clean_name)

debar_changed = debar_clean[debar_clean["Name"] != debar_clean["name_clean"]]
jobs_changed = jobs[jobs["EMPLOYER_NAME"] != jobs["name_clean"]]

In [83]:
## your code here to print the head

debar_changed[["Name", "name_clean"]]
jobs_changed[["EMPLOYER_NAME", "name_clean"]]

,Name,name_clean
3,ANTON FERTILIZER INC.,ANTON FERTILIZER INC
4,"GREAT PLAINS FLUID SERVICE, INC.","GREAT PLAINS FLUID SERVICE, INC"
5,PROMAX INC.,PROMAX INC
13,REIMER'S INC.,REIMER'S INC
19,CISCO PRODUCE INC.,CISCO PRODUCE INC
23,REIMER’S INC.,REIMER’S INC
33,GERONIMO SHEEP CO.,GERONIMO SHEEP CO
46,ALTENDORF TRANSPORT INC.,ALTENDORF TRANSPORT INC
56,CISCO PRODUCE INC.,CISCO PRODUCE INC
72,"SAXTONS RIVER ORCHARDS, INC.","SAXTONS RIVER ORCHARDS, INC"


,EMPLOYER_NAME,name_clean
4,"DUNSON HARVESTING, INC.","DUNSON HARVESTING, INC"
7,"FARM LABOR ASSOCIATION FOR GROWERS, INC.","FARM LABOR ASSOCIATION FOR GROWERS, INC"
14,"MCLAIN FARMS, INC.","MCLAIN FARMS, INC"
17,"BONNIE PLANTS, INC.","BONNIE PLANTS, INC"
18,"B & W QUALITY GROWERS, INC.","B & W QUALITY GROWERS, INC"
...,...,...
2700,HARRAL LIVESTOCK CO. LLC,HARRAL LIVESTOCK CO LLC
2701,ECOSYSTEM CONCEPTS INC.,ECOSYSTEM CONCEPTS INC
2702,SIDDOWAY SHEEP CO.,SIDDOWAY SHEEP CO
2705,SATHER MANAGEMENT INC.,SATHER MANAGEMENT INC


## 2.4 More joins and more cleaning (5 points)

A. Conduct another inner join between `jobs` and `debar_clean` now using the `name_clean` column; print the result. Did the cleaning result in any more employers matched between the two datasets?

B. Create a new column in `debar_clean` called `name_clean_2` that uses regex to take the following name in that dataset:

- `SLASH E.V. RANCH LLP` in the `debar_clean` dataset

And cleans it up so that it matches with this employer in `jobs`

- `SLASH EV RANCH` in the `jobs` dataset

Eg a pattern to remove the dots in the EV and the space+LLP-- you can apply the pattern to all employer names in debar_clean (so don't need to worry about only applying it to that one employer)


C. Conduct a left join using `name_clean_2` as the join column where the left hand dataframe is `jobs`; right hand dataframe is `debar_clean`, store the result as a dataframe, and print the rows where the merge indicator indicates the row was found in both dataframe

**Note**: this manual cleaning process is inefficient and helps motivate why talked about fuzzy matching. Fuzzy matching could recognize that Slash EV ranch is a highly similar string to slash ev ranch llp and match them without us needing to use regex to make the strings identical.

In [86]:
inner = pd.merge(jobs, debar_clean, left_on = "name_clean", right_on = "name_clean", how = "inner")
inner
# No, it did not result in any more employers matched. The number of rows (1) is still the same.

,CASE_NUMBER,CASE_STATUS,RECEIVED_DATE,DECISION_DATE,TYPE_OF_EMPLOYER_APPLICATION,H2A_LABOR_CONTRACTOR,NATURE_OF_TEMPORARY_NEED,EMERGENCY_FILING,EMPLOYER_NAME,TRADE_NAME_DBA,...,WEBSITE_TO_APPLY,TOTAL_ADDENDUM_A_RECORDS,name_clean,Name,"City, State",Violation,Duration,Start date,End date,is_repeated
0,H-300-20287-876656,Determination Issued - Certification,2020-10-20 09:20:32.010,2020-11-09 00:00:00.000,Individual Employer,Y,Seasonal,Y,RAFAEL BARAJAS,NaN,...,https://seasonaljobs.dol.gov/,7,RAFAEL BARAJAS,RAFAEL BARAJAS,"Sebring, Florida",Non-payment of certification fee,1 year,9/23/2016,9/22/2017,False


In [93]:
clean_pattern_2 = r"\.|\sLLP$"

debar_clean["name_clean_2"] = debar_clean["name_clean"].str.replace(clean_pattern_2, "", regex = True)
debar_clean[debar_clean["name_clean_2"].str.contains("SLASH", na = False)]

,Name,"City, State",Violation,Duration,Start date,End date,is_repeated,name_clean,name_clean_2
20,SLASH E.V. RANCH LLP,"Rifle, CO",WHD Debarment,1 year,11/15/2014,11/14/2015,False,SLASH E.V. RANCH LLP,SLASH EV RANCH


In [95]:
left_join = pd.merge(jobs, debar_clean, left_on = "name_clean", right_on = "name_clean_2", how = "left", indicator = True)
in_both = left_join[left_join._merge == "both"]
in_both

,CASE_NUMBER,CASE_STATUS,RECEIVED_DATE,DECISION_DATE,TYPE_OF_EMPLOYER_APPLICATION,H2A_LABOR_CONTRACTOR,NATURE_OF_TEMPORARY_NEED,EMERGENCY_FILING,EMPLOYER_NAME,TRADE_NAME_DBA,...,Name,"City, State",Violation,Duration,Start date,End date,is_repeated,name_clean_y,name_clean_2,_merge
791,H-300-20287-876656,Determination Issued - Certification,2020-10-20 09:20:32.010,2020-11-09 00:00:00.000,Individual Employer,Y,Seasonal,Y,RAFAEL BARAJAS,NaN,...,RAFAEL BARAJAS,"Sebring, Florida",Non-payment of certification fee,1 year,9/23/2016,9/22/2017,False,RAFAEL BARAJAS,RAFAEL BARAJAS,both
1115,H-300-20306-894148,Determination Issued - Certification,2020-11-02 18:11:29.140,2020-11-24 00:00:00.000,Individual Employer,N,Seasonal,N,SLASH EV RANCH,NaN,...,SLASH E.V. RANCH LLP,"Rifle, CO",WHD Debarment,1 year,11/15/2014,11/14/2015,False,SLASH E.V. RANCH LLP,SLASH EV RANCH,both


# 3. Optional extra credit 1: regex to separate companies from individuals (1 point)

You notice some employers in `debar_clean` have both the name of the company and the name of individual, e.g.:
    
COUNTY FAIR FARM (COMPANY) AND ANDREW WILLIAMSON (INDIVIDUAL)*

Use the uppercase/cleaned `name_clean` in `debar_clean`

A. Write a regex pattern that does the following:
    - Captures the pattern that occurs before COMPANY if (COMPANY) is in string; so in example above, extracts COUNTY FAIR FARM 
    - Captures the pattern that occurs before INDIVIDUAL if (INDIVIDUAL) is also in string -- so in above, extracts ANDREW WILLIAMSON (so omit the "and")
    
B. Test the pattern on `pos_example` and `neg_example`-- make sure former returns a list (if using find.all) or match object (if using re.search) with the company name and individual name separated out; make sure latter returns empty
    
**Hints and resources**: for step A, you can either use re.search, re.match, or re.findall; don't worry about matching B&R Harvesting and Paul Cruz (Individual)

- Same regex resources as above
    

C. Iterate over the `name_clean` column in debar and use regex to create two new columns in `debar_clean`:
   - `co_name`: A column for company (full `name_clean` string if no match; pattern before COMPANY if one extracted)
   - `ind_name`: A column for individual (full `name_clean` string if no match; pattern before INDIVIDUAL if one extracted)
 


   
D. Print three columns for the rows in `debar_clean` containing the negative example and positive example described above (county fair farm and cisco produce):

- `name_clean`
- `co_name`
- `ind_name`
- `Violation`

**Note**: as shown in the outcome there may be duplicates of the same company reflecting different violations

In [97]:
import re

regex = r"(.*?)(?:\s+\(COMPANY\)\s+AND\s+)(.*?)(?:\s+\(INDIVIDUAL\))?"
debar_clean[['co_name', 'ind_name']] = debar_clean['name_clean'].str.extract(regex)

debar_clean['co_name'].fillna(debar_clean['name_clean'], inplace=True)
debar_clean['ind_name'].fillna(debar_clean['name_clean'], inplace=True)

print(debar_clean[['name_clean', 'co_name', 'ind_name']])

                                            name_clean  \
0                                       J&J HARVESTING   
1                               STAHLMAN APIARIES, INC   
2                                        TRUST NURSERY   
3                                 ANTON FERTILIZER INC   
4                      GREAT PLAINS FLUID SERVICE, INC   
..                                                 ...   
108  COUNTY FAIR FARM (COMPANY) AND ANDREW WILLIAMS...   
110                                      JESUS LEDESMA   
111                                       TURNER FARMS   
112        B & R HARVESTING AND PAUL CRUZ (INDIVIDUAL)   
113                                        DELIA ROJAS   

                                         co_name  \
0                                 J&J HARVESTING   
1                         STAHLMAN APIARIES, INC   
2                                  TRUST NURSERY   
3                           ANTON FERTILIZER INC   
4                GREAT PLAINS FLUID SERVICE

/var/folders/0t/9vsbtw1s6dq13b7v62g6gwf80000gn/T/ipykernel_95293/3485666180.py:6: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  debar_clean['co_name'].fillna(debar_clean['name_clean'], inplace=True)
/var/folders/0t/9vsbtw1s6dq13b7v62g6gwf80000gn/T/ipykernel_95293/3485666180.py:7: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we a

# 4. Optional extra credit 2 (up to 3 points)

- For 1 point extra credit, create a visualization with 1+ of the existing fields in either the raw `jobs` or `debar` data. We'll be showing cool visualizations in class so use your imagination! Options could include visualizing between-state or over-time variation

- For 3 points extra credit instead, geocode the employer addresses in `jobs` and plot the addresses of jobs as points overlaid on top of a map of Georgia 
    - **Note**: this extra credit involves Googling since we have not yet covered spatial data. 
        - For discussion of how to geocode addresses -> lat/long, see: https://www.natasshaselvaraj.com/a-step-by-step-guide-on-geocoding-in-python/ 
        - For discussion of plotting lat/long dots against a map, see this discussion of geopandas: https://towardsdatascience.com/plotting-maps-with-geopandas-428c97295a73
    - Relevant columns include `EMPLOYER_ADDRESS_1` 
    - The geocoding might have a long runtime so feel free to implement it in a separate .py script that you submit alongside your notebook and to just read in the geocoded data